In [1]:
import os
import sys
import pywFM
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

NumberOfRows = 10000

os.environ['LIBFM_PATH'] = os.path.expanduser('/home/jovyan/work/libfm/bin/')
filePath = './yelp_dataset/'
pd.set_option('display.max_columns', None)
%matplotlib inline

https://github.com/jfloff/pywFM

In [2]:
business = pd.read_csv(filePath + "business_cleaned.csv", nrows=10)
user = pd.read_csv(filePath + 'user_cleaned.csv', nrows=10)
ratings = pd.read_csv(filePath + 'review_cleaned.csv', nrows=10)

In [3]:
business.head(1)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON


In [4]:
user.head(1)

,yelping_since,useful,compliment_photos,compliment_list,compliment_funny,compliment_plain,review_count,friends,fans,compliment_note,funny,compliment_writer,compliment_cute,average_stars,user_id,compliment_more,elite,compliment_hot,cool,name,compliment_profile,compliment_cool
0,2013-10-08 23:11:33,84,0,0,1,1,95,"c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",5,1,17,2,0,4.03,l6BmjZMeQD3rDxWUbiAiow,0,"2015,2016,2017",2,25,Rashmi,0,1


In [5]:
ratings.head(1)

,Unnamed: 0,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,1,NZnhc2sEQy3RmzKTZnqtwQ,0.0,2017-01-14 21:30:33,0.0,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,0.0,yXQM5uF2jS6es16SJzNHfg


In [6]:
businuessColumns = ['state', 'business_id'] #'review_count', 'stars', 
userColumns = ["user_id", "useful", "fans", "funny", "cool", "average_stars", "review_count"] #, "elite", "compliment_photos", "compliment_list", "compliment_funny", "compliment_plain", "compliment_note", "compliment_writer", "compliment_cute", "compliment_more", "compliment_hot", "compliment_profile", "compliment_cool",
ratingsColumns = ["business_id", "stars", "user_id"] #"cool", "funny", "useful", 
targetColumn = "stars"

In [7]:
business = pd.read_csv(filePath + "business_cleaned.csv", usecols=businuessColumns)
user = pd.read_csv(filePath + 'user_cleaned.csv', usecols=userColumns)
ratings = pd.read_csv(filePath + 'review_cleaned.csv', usecols=ratingsColumns, nrows=NumberOfRows)

In [8]:
print(business['business_id'].nunique())
print(business['business_id'].drop_duplicates().shape[0])

print(user['user_id'].nunique())
print(user['user_id'].drop_duplicates().shape[0])

158525
158525
818922
818922


In [9]:
# business.rename(columns={'stars': 'business_average_stars', 'review_count': 'business_review_count'}, inplace=True)
# user.rename(columns={'review_count': 'user_review_count'}, inplace=True)

In [10]:
# business[['business_review_count']].loc[business['business_review_count'] < 200].hist(bins=5)
# plt.figure(figsize=(15, 8))

# business[['business_review_count']].loc[business['business_review_count'] > 200].hist(bins=5)

In [11]:
ratings = pd.merge(ratings, business, how='left', left_on='business_id', right_on='business_id')
ratings = pd.merge(ratings, user, how='left', left_on='user_id', right_on='user_id')

In [12]:
ratings.shape

(10000, 10)

In [13]:
ratings['business_id'] = ratings['business_id'].astype('category')
ratings['business_id'] = ratings['business_id'].cat.codes

ratings['user_id'] = ratings['user_id'].astype('category')
ratings['user_id'] = ratings['user_id'].cat.codes

In [14]:
ratings.head()

,business_id,stars,user_id,state,useful,review_count,fans,funny,average_stars,cool
0,1577,5.0,9111,NV,178,68,3,36,3.97,67
1,2151,5.0,7480,AZ,13,10,1,1,4.27,1
2,289,3.0,6987,ON,4346,600,83,2302,3.75,4067
3,2315,5.0,3657,ON,2,9,0,1,2.30,0
4,1603,4.0,1350,OH,3,14,0,1,4.00,3


In [15]:
def convertDummies(df, column_name):
    df = pd.concat([
        df.drop(column_name, axis=1),
        pd.get_dummies(df[column_name], prefix=column_name+'_')
    ], axis=1)
    print(column_name, ' is ready.')
    return df

In [16]:
ratings = convertDummies(ratings, 'business_id')
ratings = convertDummies(ratings, 'user_id')
ratings = convertDummies(ratings, 'state')

business_id  is ready.
user_id  is ready.
state  is ready.


In [ ]:
ratings.head()

In [17]:
train_X = ratings.drop(targetColumn, axis=1).iloc[:-2000]
train_y = ratings[targetColumn].iloc[:-2000]

test_X = ratings.drop(targetColumn, axis=1).iloc[-2000:]
test_y = ratings[targetColumn].iloc[-2000:]

In [25]:
fm = pywFM.FM(task='regression')
model = fm.run(train_X.values, train_y.values, test_X, test_y)

In [26]:
pd.concat([test_y.reset_index(drop=True), pd.Series(model.predictions, name='predicted')], axis=1).head()

,stars,predicted
0,5.0,1.64
1,4.0,1.28
2,5.0,1.56
3,1.0,1.60
4,1.0,1.52


In [27]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(test_y.values, model.predictions))

2.6375188272475514

In [28]:
np.sqrt(mean_squared_error(test_y.values, np.tile(ratings['stars'].mean(), test_y.shape[0])))

1.442641660288514